In [1]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('/home/agnel/Agnel/files/HaarCascade/haarcascade_frontalface_alt2.xml')

cap = cv2.VideoCapture(0)
i = 0
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        #print(x, y, w, h)
        img_item = str(i)+".png"
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.imwrite(img_item, roi_color)
        
        color= (0, 0, 255) #BGR
        stroke = 2
        width = x + w
        height = y + h
        cv2.rectangle(frame, (x - 10, y), (width + 10, height + 20), color, stroke)
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == 13:
        break
    
cap.release()
cv2.destroyAllWindows()
    

error: OpenCV(4.0.0) C:\projects\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1658: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'


# Face Train

In [3]:
import os
from setuptools import setup

BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
# print(BASE_DIR)
img_dir = os.path.join(BASE_DIR, "images")
# print(img_dir)

##Loading images and cropping the image
# i = 0
# for root, dirs, files in os.walk(img_dir):
#     for file in files:
#         if file.endswith("png") or file.endswith("jpg"):
#             path = os.path.join(root, file)
#             img = cv2.imread(path)
#             img = cv2.resize(img, (280, 280))
#             gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#             faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#             for (x, y, w, h) in faces:
#                     img_name = str(i)+".png"
#                     roi_color = img[y:y+h+20, x:x+w+20]
#                     cv2.imwrite(img_name, roi_color)
#                     #cv2.imshow("Frame",roi_color)
#                     i += 1
#             cv2.waitKey(0)
# cv2.destroyAllWindows()

##Training image from cropped image
from PIL import Image
import numpy as np

current_ids = 0
label_ids = {}
x_train = []
y_labels = []
for root, dirs, files in os.walk(img_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(' ', "-")
            if label in label_ids:
                pass
            else:
                label_ids[label] = current_ids
                current_ids += 1
            id_ = label_ids[label]
            #print(label_ids)
            pil_image = Image.open(path).convert("L")
            image_array = np.array(pil_image, 'uint8')
            #print(image_array)
            faces = face_cascade.detectMultiScale(image_array, 1.5, 5)
            
            for(x, y, w, h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)

# print(x_train)
# print(y_labels)

#############Saving the model and label using pickle ##############
import pickle

recognizer = cv2.face.LBPHFaceRecognizer_create()

with open("label.pickle", "wb") as f:
    pickle.dump(label_ids, f)
    
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainner.yml")
                
                

error: OpenCV(4.0.0) C:\projects\opencv-python\opencv_contrib\modules\face\src\lbph_faces.cpp:362: error: (-210:Unsupported format or combination of formats) Empty training data was given. You'll need more than one sample to learn a model. in function 'cv::face::LBPH::train'


# Now Using the trained model


In [14]:
import pickle
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('/home/agnel/Agnel/files/HaarCascade/haarcascade_frontalface_alt2.xml')
eye_cascade = cv2.CascadeClassifier('/home/agnel/Agnel/files/HaarCascade/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('/home/agnel/Agnel/files/HaarCascade/haarcascade_smile.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainner.yml")

labels = {}
with open('label.pickle', 'rb') as f:
    og_labels = pickle.load(f)
    labels = {v:k for k, v in og_labels.items()}
stroke = 2
cap = cv2.VideoCapture(0)
i = 0
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        #print(x, y, w, h)
        img_item = str(i)+".png"
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        id_, conf = recognizer.predict(roi_gray)
        #print(labels[id_])
        cv2.putText(frame, labels[id_], (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), stroke)
                
        color= (0, 0, 255) #BGR
        stroke = 2
        width = x + w
        height = y + h
        cv2.rectangle(frame, (x - 10, y), (width + 10, height + 20), color, stroke)
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0))
        #smile = smile_cascade.detectMultiScale(roi_gray)
        #for (sx, sy, sw, sh) in smile:
            #cv2.rectangle(roi_color, (sx, sy), (sx+sw, sy+sh), (255, 0, 0))
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == 13:
        break
    
cap.release()
cv2.destroyAllWindows()

